<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Двойной Маятник и Хаотическое движение

### Examples - Mechanics
<section class="post-meta">
By Jonas Tjemsland, Eilif Sommer Øyre and Jon Andreas Støvneng
</section>
Last edited: April 15th 2018 
___

Простые нелинейные дифференциальные уравнения могут иметь сложные и хаотичные решения. В этом блокноте мы будем изучать хаотическое движение двойного маятника. Несмотря на то, что движение хаотично, графики фазового пространства показывают прекрасный порядок, лежащий в основе хаоса.

Двойной маятник показан на рисунке [1](#im1). Он состоит из двух простых маятников, где масса первого маятника является точкой поворота для второго. Мы отсылаем вас к нашему блокноту о [простом маятнике](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/simple_pendulum.ipynb) для краткого ознакомления. Пусть $m_i$ и $L_i$ - масса грузика и длина простого маятника $i$ соответственно. $\theta_1$ - это угол между вертикальным направлением и первым маятником, а $\theta_2$ - это угол между вертикальным направлением и вторым, как показано на рисунке. Обратите внимание, что существует только две степени свободы: $\theta_1$ и $\theta_2$. Однако этого достаточно, чтобы получить хаотическое движение.

<a id="im1"></a>
<br />
![Double Pendulum](images/double_pendulum.png)
**Figure 1.** Двойной маятник с массами $m_1$ и $m_2$, углами смещения $\theta_1$ и $\theta_2$ и длинами $L_1$ и $L_2$.
<br></br>

Мы начинаем с установки некоторых констант построения графика и импорта необходимых пакетов.

In [ ]:
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from numpy import cos, sin, arange, pi
import matplotlib.cm as cm
%matplotlib inline

figsize = 6
dpi = 600
g = 9.81     # [m/s^2]. Гравитационное ускорение

## Движение двойного маятника

Уравнения движения можно записать в виде (вывод см. в [приложении](#apndx))

\begin{align}
\dot\omega_1 &= \frac{1}{L_1\xi}\left[L_1m_2c_{12}s_{12}\omega_1^2 + L_2m_2s_{12}\omega_2^2 - m_2gc_{12}s_2 + (m_1+m_2)gs_1 \right],\\
\dot\omega_2 &= \frac{1}{L_2\xi}\left[L_2m_2c_{12}s_{12}\omega_2^2 + L_1(m_1+m_2)s_{12}\omega_1^2+(m_1+m_2)gs_1c_{12} - (m_1+m_2)gs_2 \right],\\
\omega_1 &= \dot\theta_1,\\
\omega_2 &= \dot\theta_2,
\end{align}

где мы определили $c_{12} \equiv \cos(\theta_1-\theta_2)$, $s_{12} \equiv \sin(\theta_1-\theta_2)$, $s_i \equiv \sin(\theta_i)$ и $\xi \equiv c_{12}^2m_2-m_1-m_2$ для упрощения обозначения. Обратите внимание, что мы записали уравнения в виде четырех связанных обыкновенных дифференциальных уравнений первого порядка, что означает, что мы можем легко решить соответствующую начальную задачу, используя, например, метод Эйлера или метод Рунге-Кутты 4-го порядка. В этом блокноте мы будем использовать интегратор `odeint` от `scipy`. Эта функция решает проблему начального значения для жестких или нежестких систем ОДУ первого порядка. Почитайте [[1]](#rsc), чтобы узнать, как он работает. Чтобы использовать `odeint`, нам нужно создать функцию, которая вычисляет правую часть уравнений движения. Также будет удобно определить функцию, которая преобразует $\omega_1$,$\omega_2$, $\theta_1$ и $\theta_2$ в декартовы координаты $\mathbf{v}_1$, $\mathbf{v}_2$, $\mathbf{x}_1$ и $\mathbf{x}_2$.

In [ ]:
def RHS(z, t, L1, L2, m1, m2, g):
    """ Возвращает правую часть
        обыкновенного дифференциального уравнения, описывающего
        двойной маятник.
    """
    theta1, w1, theta2, w2 = z
    cos12 = cos(theta1 - theta2)
    sin12 = sin(theta1 - theta2)
    sin1 = sin(theta1)
    sin2 = sin(theta2)
    xi = cos12**2*m2 - m1 - m2
    w1dot = ( L1*m2*cos12*sin12*w1**2 + L2*m2*sin12*w2**2
            - m2*g*cos12*sin2      + (m1 + m2)*g*sin1)/(L1*xi)
    w2dot = -( L2*m2*cos12*sin12*w2**2 + L1*(m1 + m2)*sin12*w1**2
            + (m1 + m2)*g*sin1*cos12  - (m1 + m2)*g*sin2 )/(L2*xi)
    return w1, w1dot, w2, w2dot

def to_cartesian(theta1, w1, theta2, w2, L1, L2):
    """ Преобразует тета и омега в декартовы координаты
    и скорости x1, y1, x2, y2, vx1, vy1, vx2, vy2
    """
    x1 = L1 * sin(theta1)
    y1 = -L1 * cos(theta1)
    x2 = x1 + L2 * sin(theta2)
    y2 = y1 - L2 * cos(theta2)
    vx1 = L1*cos(theta1)*w1
    vy1 = L1*sin(theta1)*w1
    vx2 = vx1 + L2*cos(theta2)*w2
    vy2 = vy1 + L2*sin(theta2)*w2
    return x1, y1, x2, y2, vx1, vy1, vx2, vy2

Теперь нам нужно определить параметры. Поиграйте с начальными условиями и различными параметрами!

Здесь мы делаем (произвольный) выбор $2L_1=L_2$ и $m_1=3m_2$, и мы позволим маятнику качаться в течение 50 секунд.

In [ ]:
L1, L2 = 1., 2.
m1, m2 = 3., 1.

z0 = [pi/2, 0, pi/2, 0]
tmax, dt = 50, 0.01
t = arange(0, tmax+dt, dt)

Теперь мы готовы выполнить моделирование и преобразовать результаты в декартовы координаты.

In [ ]:
# Выполнит моделирование
z = odeint(RHS, z0, t, args=(L1, L2, m1, m2, g))

# Извлечь результат
theta1, w1, theta2, w2 = z[:,0], z[:,1], z[:,2], z[:,3]
x1, y1, x2, y2, vx1, vy1, vx2, vy2 = to_cartesian(theta1, w1, theta2, w2, L1, L2)

Теперь мы можем построить график результатов и создать анимацию! С этой целью мы создали функции `plot_position()` и `create_animation()`, которые показаны в [приложении](#apndx).

In [ ]:
plot_position(x1, y1, x2, y2, theta1, theta2, t)

In [ ]:
create_animation("double_pendulum", x1, y1, x2, y2, tmax, L1, L2)

Как видно, результат довольно хаотичен и трудно поддается интерпретации. Часто такие сложные функции времени кажутся более простыми в так называемых *диаграммах фазового пространства* [[2]](#rsc). Мы отложим дальнейшие обсуждения движения до тех пор, пока не вычислим диаграмму фазового пространства.

## Диаграммы фазового пространства для простого маятника

Прежде чем анализировать фазовые диаграммы для двойного маятника, мы хотим обсудить оные для более простой и упорядоченной системы. В качестве примера мы рассмотрим простой маятник, обсуждаемый в нашем блокноте про [Простой маятник](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/simple_pendulum.ipynb).

In [ ]:
def RHS_simple(z, t, L, g):
    """ Возвращает правую часть
    обыкновенного дифференциального уравнения, описывающего
    простой маятник.
    """
    theta, w = z
    wdot = -sin(theta)*g/L
    return w, wdot

Общим решением уравнения движения является "положение" $x(t)$ и "скорость" $v(t)$. Для простого маятника это соответствует $\theta(t)$ и $\omega(t)$. Участок $(\theta(t), \omega(t))$ известен как участок *фазового пространства*. Давайте создадим график фазового пространства для простого маятника для нескольких начальных условий, а затем интерпретируем и обсудим результаты. Для простоты мы позволим $\theta_0=0$ и $\omega_0$ варьироваться.

In [ ]:
w0 = [2., 4., 6, 6.2, 6.3, 6.4, 7.]
L = 1.
g = 9.81
tmax_simp, dt_simp = 4, 0.01
t_simp = arange(0, tmax+dt, dt)

plt.figure(figsize=(2*figsize, figsize), dpi=dpi)
plt.title(r"Phase-space diagram, simple pendulum, $\theta_0=0$")
for i in range(len(w0)):
    z = odeint(RHS_simple, [0, w0[i]], t_simp, args=(L, g))
    theta_simp, w_simp = z[:,0], z[:,1]
    color = cm.rainbow(float(i + 1)/len(w0)) # Creating colormap
    plt.plot(theta_simp, w_simp, c=color, label=r"$\omega_0=%.1fs^{-1}$"%(w0[i]))
    plt.plot(theta_simp, -w_simp, c=color)
    plt.plot(-theta_simp, w_simp, c=color)
    plt.plot(-theta_simp, -w_simp, c=color)
plt.xlabel(r"$x(t)/L$")
plt.ylabel(r"$v(t)/(L/s)$")
plt.xlim([-pi, 2*pi])
plt.legend()
plt.show()

Хотя диаграмма фазового пространства не показана полностью, она периодична с периодом $2\pi$. Обратите внимание на следующее (см., например, [[2]](#rsc) для получения дополнительной информации):

- Для гармонического потенциала диаграммы фазового пространства становятся эллипсами (см. упражнение ниже). Для малого $\theta_\text{max}$ и, следовательно, малого $\omega_0$ потенциал простого маятника приблизительно гармонический, и диаграммы фазового пространства, таким образом, являются эллипсами для малого $\omega_0$.
- При увеличении $\omega_0$ орбиты становятся более ангармоническими и при $\theta(t)=\pm\pi$ будут наблюдаться более пологие линии диаграммы.
- Замкнутые орбиты соответствуют периодическим колебаниям, в то время как открытые орбиты соответствуют непериодическому (или "бегущему") движению.
- Если бы мы включили трение, например, для затухающего маятника, орбита фазового пространства закрутится в неподвижную точку (превратится в спираль).
- Разные орбиты не пересекаются, так как решение для разных начальных условий уникально.

**Упражнение:** Докажите, что диаграммы фазового пространства для гармонических потенциалов действительно являются эллипсами. *Подсказка: Вставьте уравнение движения для простого маятника с $\theta_0\ll 1$ в выражение для полной механической энергии.*  
**Упражнение:** Постройте диаграмму фазового пространства для затухающего простого маятника.

## Диаграммы фазового пространства для двойного маятника

Движения двойного маятника гораздо сложнее, чем у простого маятника. Для удобства мы будем рассматривать только движение массы $m_2$. Давайте построим диаграмму фазового пространства для двойного маятника, обсуждавшегося ранее. С этой целью мы будем использовать функцию `plot_phasespace()`, определенную в [приложении](#apndx).

In [ ]:
plot_phasespace(theta1, w1, theta2, w2)

Опять же, движение двойного маятника явно сложнее, чем для простого маятника. Однако некоторые характеристики структуры из диаграммы фазового пространства для простого маятника могут быть распознаны. Это легче всего увидеть на орбите фазового пространства для $m_1$. Здесь мы ясно видим, что существует несколько "эллиптических" периодических структур. Траектории не подчинены единой структуре, но в некотором смысле стремятся к ней. "Случайность" приводит к структурам, состоящим из диффузных полос, а не одиночных линий, полученных в случае простого маятника. Это то, что характеризует "хаотическое движение". Это также является причиной того, что движение системы сверхчувствительно к изменению начальных условий (попробуйте сами!).

Обратите внимание, что движение массы $m_2$ имеет большую тенденцию к вращению, чем $m_1$. Другими словами, $\theta_1$ увеличивается быстрее, чем $\theta_2$ в зависимости от времени. В этом смысле движение массы $m_1$ является более "случайным".

Поиграйте с другими начальными условиями!

## Вывод
В этом блокноте хаотическое движение вводится с помощью двойного маятника в качестве примера. Несмотря на то, что движение системы хаотично, существуют четкие структуры фазового пространства. Это означает, что в хаосе есть скрытый порядок.

## Дальнейшая Работа
- Движение является хаотичным не для всех начальных условий. Найдите несколько примеров.
- Реализуйте "управляемый демпфированный маятник" и выполните тот же анализ, что и выше. В этой системе энергия может рассеиваться, и могут возникнуть другие интересные явления, такие как *предсказуемые аттракторы* и *блокировка режима*.
- Рассматриваемая здесь система имеет четырехмерное пространство параметров $(\theta_1, \theta_2, \omega_1, \omega_2)$. Его можно изучить более подробно.
- Используйте анализ Фурье для анализа хаоса.

<a id="rsc"></a>
## Литература
<a>[1]</a>: `scipy.integrate.odeint`, [scipy.org](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html).  
<a>[2]</a>: Landau, R. H., Páez, M. J. and Bordeianu, C. C.: Computational Physics, 2nd edition, WILEY-VCH 2007.

___

<a id="apndx"></a>
# Appendix

### Уравнения движения
Рассмотрим установку на рисунке [1](#im1). Положение массы $m_1$ определяется

\begin{equation}
\begin{aligned}
x_1 &= L_1\sin\theta_1\\
y_1 &= -L_1\cos\theta_1.
\label{eq:pos1}\quad(1)
\end{aligned}
\end{equation}

Положение массы $m_2$ может быть выражено как

\begin{equation}
\begin{aligned}
x_2 &= x_1 + L_2\sin\theta_2\\
y_2 &= y_1 -L_2\cos\theta_2.
\label{eq:pos2}\quad(2)
\end{aligned}
\end{equation}

Используя $v_1^2 = \dot x_1^2 +\dot y_1^2$ и $v_2^2 = \dot x_2^2 +\dot x_2^2$ и уравнения (1) и (2), легко показать, что кинетические энергии масс имеют вид

\begin{equation}
\begin{aligned}
T_1 &= \frac{1}{2}m_1v_1^2 = \frac{1}{2}m_1L_1^2\dot\theta_1^2,\\
T_2 &= \frac{1}{2}m_2v_2^2 = \frac{1}{2}m_2\left(L_1^2\dot\theta_1^2 + L_2^2\dot\theta_2^2+2L_1L_2\cos(\theta_1-\theta_2)\dot\theta_1\dot\theta_2\right).
\end{aligned}
\label{eq:T}\quad(3)
\end{equation}

а потенциальные

\begin{equation}
\begin{aligned}
V_1 &= m_1gy_1 = -m_1gL_1\cos\theta_1,\\
V_2 &= m_2gy_2 = -m_2g\left(L_1\cos\theta_1+L_2\cos\theta_2\right).
\end{aligned}
\label{eq:V}\quad(4)
\end{equation}

Теперь мы формируем лагранжиан $L=T-V = T_1+T_2-V_1-V_2$ из уравнений (3) и (4) и вставляем его в уравнения Эйлера-Лагранжа

\begin{equation}
\frac{\partial L}{\partial \theta_1} = \frac{\mathrm{d}}{\mathrm{d}t}\frac{\partial L}{\partial \dot\theta_1}
\label{eq:EL1}\quad(5)
\end{equation}

и

\begin{equation}
\frac{\partial L}{\partial \theta_2} = \frac{\mathrm{d}}{\mathrm{d}t}\frac{\partial L}{\partial \dot\theta_2}.
\label{eq:EL2}\quad(6)
\end{equation}

Уравнение (5) дает

\begin{equation}
-m_1gL_1\sin\theta_1-m_2L_1g\sin\theta_1 = (m_1+m_2)L_1^2\ddot\theta_1 + m_2L_1L_2\sin(\theta_1-\theta_2)\dot\theta^2 + m_2L_1L_2\cos(\theta_1 - \theta_2)\ddot\theta_2,
\end{equation}

в то время как уравнение (6) дает

\begin{equation}
-m_2gL_2\sin\theta_2 = m_2L_2^2\ddot\theta_2 + m_2L_1L_2\cos(\theta_1-\theta_2)\ddot\theta_1-m_2L_1L_2\sin(\theta_1-\theta_2)\dot\theta_1^2.
\end{equation}

Если мы решим их по $\ddot\theta_1$ и $\ddot\theta_2$, мы получим после некоторых манипуляций

\begin{equation}
\begin{aligned}
\ddot\theta_1 &= \frac{1}{L_1\xi}\left[L_1m_2\cos(\theta_1-\theta_2)\sin(\theta_1-\theta_2)\dot\theta_1^2 + L_2m_2\sin(\theta_1-\theta_2)\dot\theta_2^2 - m_2g\cos(\theta_1-\theta_2)\sin(\theta_2) + (m_1+m_2)g\sin(\theta_1) \right],\\
\ddot\theta_2 &= \frac{1}{L_2\xi}\left[L_2m_2\cos(\theta_1-\theta_2)\sin(\theta_1-\theta_2)\dot\theta_2^2 + L_1(m_1+m_2)\sin(\theta_1-\theta_2)\dot\theta_1^2+(m_1+m_2)g\sin(\theta_1)\cos(\theta_1-\theta_2) - (m_1+m_2)g\sin(\theta_2) \right],\\
\end{aligned}
\label{eq:motion_1}\quad(7)
\end{equation}

\begin{equation}
\xi \equiv \cos^2(\theta_1-\theta_2)m_2-m_1-m_2.
\end{equation}

Система (7) - это уравнения движения для двойного маятника. Удобно ввести $\omega_i\equiv \dot\theta_i$ и переформулировать уравнения движения в виде четырех дифференциальных уравнений первого порядка.

### Функции построения графиков


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image
import matplotlib.animation as animation

def plot_position(x1, y1, x2, y2, theta1, theta2, t):
    """ Plots the motion of the double pendulum in the
    xy-plane, as well as the angles and the angular
    velocities as a function of time.
    
    x1     : array-like, size(n,). x-posision of mass 1
    y1     : array-like, size(n,). y-posision of mass 1
    x2     : array-like, size(n,). x-posision of mass 2
    y2     : array-like, size(n,). y-posision of mass 2
    theta1 : array-like, size(n,). The first angle in the double pendulum
    theta2 : array-like, size(n,). The second angle in the double pendulum
    t      : array-like, size(n,). Time
    """
    plt.figure(figsize=(2*figsize, figsize), dpi=dpi)

    # xy-plot
    L = 1.1*(L1 + L2)
    ax = plt.subplot(2, 2, (1, 3), autoscale_on=False, xlim=(-L, L), ylim=(-L, L))
    ax.plot(x1, y1, label=r"Track $m_1$")
    ax.plot(x2, y2, label=r"Track $m_2$")
    ax.plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", label="Initial position", c='k')
    plt.ylabel(r"$y/L$")
    plt.xlabel(r"$x/L$")
    ax.legend()

    # xt-plot
    ax = plt.subplot(2, 2, 2)
    ax.plot(t, theta1, label=r"$\theta_1(t)$")
    ax.plot(t, theta2, label=r"$\theta_2(t)$")
    plt.ylabel(r"$\theta$, [rad]")
    plt.xlabel(r"$t$, [s]")
    ax.legend()
    plt.xlim([0, np.max(t)])

    # yt-plot
    ax = plt.subplot(2, 2, 4)
    ax.plot(t, w1, label=r"$\omega_1(t)$")
    ax.plot(t, w2, label=r"$\omega_2(t)$")
    plt.ylabel(r"$\omega$, [rad/s]")
    plt.xlabel(r"$t$, [s]")
    plt.xlim([0, np.max(t)])
    ax.legend()

    plt.show()
    
def create_animation(filename, x1, y1, x2, y2, tmax, L1, L2):
    """ Creates an animation of the double pendulum.
    The animation is stored as a .gif and displayed.
    
    filename : string. Filename of the animation
    x1     : array-like, size(n,). x-posision of mass 1
    y1     : array-like, size(n,). y-posision of mass 1
    x2     : array-like, size(n,). x-posision of mass 2
    y2     : array-like, size(n,). y-posision of mass 2
    tmax   : array-like, size(n,). Final time
    L1, L2 : float. Length of rod 1 and 2
    """
    
    fig = plt.figure(figsize=(4, 4), dpi=60)
    L = 1.1*(L1 + L2)
    ax = fig.add_subplot(111, autoscale_on=False, xlim=(-L, L), ylim=(-L, L))

    tail1, = ax.plot([],[],'r') # Tail for m2
    line, = ax.plot([], [], '-o', lw=2, c="k")
    time_template = r'$t = %.1fs$'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    ax.set_aspect('equal')
    ax.axis('off')

    # Calculates the number of frames
    FPS = 15
    framesNum = int(FPS*tmax)
    frames = np.floor(np.linspace(0, len(x1) - 1, framesNum)).astype(np.int)

    def init():
        line.set_data([], [])
        tail1.set_data([], [])
        time_text.set_text('')
        return line, time_text

    def animate(i):
        line.set_data([0, x1[i], x2[i]], [0, y1[i], y2[i]])
        tail1.set_data(x2[:i], y2[:i])
        time_text.set_text(time_template % (i*dt))
        return line, time_text, tail1

    anim = animation.FuncAnimation(fig, animate, frames=frames)

    # Save animation. Try using "ffmpeg" if "imagemagick" does not work
    anim.save(filename + '.gif', writer='imagemagick', fps=FPS)

    # Close plot
    plt.close(anim._fig)

    # Display the animation
    with open(filename + '.gif','rb') as file:
        display(Image(file.read()))
        
def plot_phasespace(theta1, w1, theta2, w2):
    """ Creates a phase-space plot for the double pendulum
    for (theta, omega).
    
    theta1 : array-like, size(n,). The first angle in the double pendulum
    w1     : array-like, size(n,). Angular velocity of the first angle
    theta2 : array-like, size(n,). The second angle in the double pendulum
    w2     : array-like, size(n,). Angular velocity of the second angle
    """
    
    plt.figure(figsize=(2*figsize, figsize), dpi=dpi)
    plt.title(r"Phase-space diagram, $\theta_{10}=%.1f$, $\theta_{20}=%.1f$ "%(theta1[0], theta2[0])
             + r"$\omega_{10}=%.1f$, $\omega_{20}=%.1f$"%(w1[0], w2[0]))
    plt.plot(theta1, w1, label=r"$i=1$")
    plt.plot(theta2, w2, label=r"$i=2$")
    plt.legend()
    plt.xlabel(r"$\theta_i$, [rad]")
    plt.ylabel(r"$\omega_i$, [rad/s]")
    xlim = [np.min(theta1), np.max(theta1), np.min(theta2), np.max(theta2)]
    plt.xlim(np.min(xlim), np.max(xlim))
    plt.show()